In [1]:
# Imports
import pathlib
import os
from dataclasses import dataclass, field
from typing import Dict, Literal, Optional, Tuple
import json

import imutils
import numpy as np
import cv2
from dataclasses_json import DataClassJsonMixin
import pandas as pd
from tqdm import tqdm

# Constants 
CWD = pathlib.Path(os.path.abspath(""))
GIT_ROOT = CWD.parent
DATA_DIR = GIT_ROOT / "data" / 'PhotosynthesisFall2022'

In [27]:
# Game state reconstruction routines
@dataclass
class Participant(DataClassJsonMixin):
    id: str
    position: Tuple[float, float]
    state: Literal['null', 'H2O', 'CO2', 'Sugar', 'O2', 'Thinking_H2O'] = 'null'

@dataclass
class EnvironmentState(DataClassJsonMixin):
    sun_state: Optional[bool] = None

@dataclass
class GameState(DataClassJsonMixin):
    participants: Dict[str, Participant] = field(default_factory=dict)
    environment: EnvironmentState = field(default_factory=EnvironmentState)

@dataclass
class GamePhaseState(DataClassJsonMixin):
    phase: Literal['before_game', 'molecule_select', 'normal', 'paused']

In [3]:
# Helper functions
def xy_transforms(xy: Tuple[float, float], w: int, h:int, c: Dict[str, float]) -> Tuple[float, float]:
    x, y = xy
    xx = int(((x+1)*h/2)*c['AFFINE'][0] + c['OFFSET'][0])
    yy = int(((y+1)*w/2)*c['AFFINE'][1] + c['OFFSET'][1])
    return (xx, yy)

def render(game_state: GameState, frame: np.ndarray, c: Dict[str, float]):

    if game_state.participants:
        for p in game_state.participants.values():
            if isinstance(p.position[0], str):
                p.position = (eval(p.position[0]), eval(p.position[1]))
            xy = xy_transforms(p.position, frame.shape[0], frame.shape[1], c)
            frame = cv2.circle(frame, xy, 30, (0, 0, 255), 1)
            frame = cv2.putText(
                frame, 
                p.id, 
                (xy[0]-22, xy[1]-11), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (0,0,0), 
                2, 
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.id, 
                (xy[0]-22, xy[1]-11), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (255,255,255), 
                1, 
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.state, 
                (xy[0]-5*(len(p.state)), xy[1]+13), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, 
                (0,0,0), 
                2,
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.state, 
                (xy[0]-5*(len(p.state)), xy[1]+13), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, 
                (255,255,255), 
                1,
                cv2.LINE_AA
            )

    return frame

In [41]:
# Load the data
vid_file = DATA_DIR / 'videos' / 'day 13' / "day13-screen-recording-corrected.mp4"
assert vid_file.exists()
cap = cv2.VideoCapture(str(vid_file))
fps = cap.get(cv2.CAP_PROP_FPS)

game_state_file = DATA_DIR / 'time_alignment' / 'day13_aligned_game_state.csv'
assert game_state_file.exists()
game_state_logs = pd.read_csv(game_state_file)

annotated_events_file = DATA_DIR / 'videos' / 'day 13' / 'annotated_game_events.csv'
assert annotated_events_file.exists()
annotated_events = pd.read_csv(annotated_events_file)

# Game logs
TIME_OFFSET = 4*60 + 30.9
game_logs = DATA_DIR / 'logs' / 'VU_GEM-STEP_NB_2022_Fall_AH_GroupB_Day13_221128_ComputerLogs' / 'game_logs.csv'
assert game_logs.exists()
game_logs = pd.read_csv(game_logs)
game_logs = game_logs[game_logs['event_type'] == 'touch'].reset_index(drop=True)
game_logs['datetime'] = pd.to_datetime(game_logs['datetime'], format='%Y-%m-%d %H:%M:%S.%f')
game_logs['timestamp'] = (game_logs['datetime'] - game_logs['datetime'].iloc[0]).dt.total_seconds()
game_logs['timestamp'] = game_logs['timestamp'] - TIME_OFFSET
game_logs = game_logs[game_logs['timestamp'] >= 0].reset_index(drop=True)

# Combine game and annotated events
game_logs = pd.concat([game_logs, annotated_events])
game_logs = game_logs.sort_values('timestamp')

In [50]:
# Transformation constants
START_INDEX = 1*60*fps+58*fps
CORRECTIONS = {'OFFSET': (-252,-280), 'AFFINE': (1.8,2)}
N = 20_000
sprite_map = {
    'molecules': {
        '32': 'H2O',
        '33': 'CO2',
        '34': 'Sugar',
        '35': 'O2',
    },
    'actors': {
        '24': 'plant_body',
        '25': 'root',
        '26': 'rabbit',
        '46': 'leaf',
        '47': 'leaf'
    }
}

# Reset video and logs
cap.set(cv2.CAP_PROP_POS_FRAMES, START_INDEX)
game_state_pointer = 0
game_event_pointer = -1
game_state = None
game_phase = GamePhaseState(phase='before_game')
molecule_select_time = None

for i in tqdm(range(N), total=N):
    
    ret, frame = cap.read()
    if not ret:
        break

    frame_timestamp = (START_INDEX + i) / fps
    frame = cv2.putText(
        frame, 
        f"{pd.Timestamp(frame_timestamp, unit='s').strftime('%H:%M:%S.%f')}", 
        (5,30), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        1, 
        (0,0,255), 
        1, 
        cv2.LINE_AA
    )

    # Obtaining game state
    while game_state_logs['timestamp'].iloc[game_state_pointer+1] < frame_timestamp:
        game_state_pointer += 1

    new_game_state = GameState.from_json(game_state_logs.iloc[game_state_pointer].state)
    if game_state:
        for p in game_state.participants.values():
            if p.id in new_game_state.participants:
                p.position = new_game_state.participants[p.id].position
    else:
        game_state = new_game_state

    # Processing events
    while len(game_logs) > game_event_pointer+1 and game_logs['timestamp'].iloc[game_event_pointer+1] < frame_timestamp:

        event = game_logs.iloc[game_event_pointer+1]
        if event.event_type == 'game_reset':
            game_phase.phase = 'before_game'
            for p in game_state.participants.values():
                p.state = 'O2'

        elif event.event_type == 'phase':
            data = event.event_data.replace("“", '"')
            data = data.replace("”", '"')
            phase = json.loads(data)['phase']
            assert phase in ['before_game', 'molecule_select', 'normal', 'paused']
            game_phase.phase = phase

            if phase == 'molecule_select':
                molecule_select_time = event.timestamp

        elif event.event_type == 'touch':
            data = json.loads(event.event_data)
            print(data)
            if 'pz' in data['src']:
                if game_phase.phase in ['before_game', 'paused']: # nothing
                    ...
                elif game_phase.phase == 'molecule_select': # permit transition to molecules
                    if data['dst'] in sprite_map['molecules']:
                        p = game_state.participants[data['src']]
                        p.state = sprite_map['molecules'][data['dst']]
                elif game_phase.phase == 'normal': # Only allow photosynthesis transitions
                    ...

        game_event_pointer += 1

    # For macro alignment
    # frame = cv2.putText(
    #     frame, 
    #     f"{game_state_pointer}", 
    #     (5,60), 
    #     cv2.FONT_HERSHEY_SIMPLEX,
    #     1, 
    #     (0,0,255), 
    #     1, 
    #     cv2.LINE_AA
    # )
    frame = cv2.putText(
        frame, 
        f"{game_phase.phase}", 
        (5,60), 
        cv2.FONT_HERSHEY_SIMPLEX,
        1, 
        (0,0,255), 
        1, 
        cv2.LINE_AA
    )

    frame = render(game_state, frame, CORRECTIONS)

    cv2.imshow('frame', imutils.resize(frame, width=1000))
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

  0%|          | 0/20000 [00:00<?, ?it/s]

{'src': 'pz6953', 'dst': '24'}
{'src': 'pz6938', 'dst': '35'}
{'src': '35', 'dst': 'pz6938'}
{'src': 'pz6951', 'dst': '24'}
{'src': 'pz6942', 'dst': '24'}
{'src': 'pz6915', 'dst': '24'}
{'src': 'pz6954', 'dst': '24'}
{'src': 'pz219', 'dst': '24'}
{'src': '35', 'dst': 'pz6938'}
{'src': 'pz6951', 'dst': '24'}
{'src': 'pz6954', 'dst': '24'}
{'src': 'pz6953', 'dst': '24'}
{'src': 'pz6915', 'dst': '24'}
{'src': 'pz219', 'dst': '24'}
{'src': 'pz6938', 'dst': '35'}
{'src': 'pz6942', 'dst': '24'}
{'src': 'pz6954', 'dst': '24'}
{'src': 'pz6915', 'dst': '24'}
{'src': 'pz6938', 'dst': '35'}
{'src': 'pz6942', 'dst': '24'}
{'src': 'pz219', 'dst': '24'}
{'src': '35', 'dst': 'pz6938'}
{'src': 'pz6951', 'dst': '24'}
{'src': 'pz6953', 'dst': '24'}
{'src': 'pz6953', 'dst': '24'}
{'src': 'pz6938', 'dst': '35'}
{'src': '35', 'dst': 'pz6938'}
{'src': 'pz6951', 'dst': '24'}
{'src': 'pz6954', 'dst': '24'}
{'src': 'pz219', 'dst': '24'}
{'src': 'pz6942', 'dst': '24'}
{'src': 'pz6915', 'dst': '24'}
{'src': '35'

  0%|          | 2/20000 [00:39<93:13:45, 16.78s/it] 

{'src': 'pz6951', 'dst': '24'}
{'src': 'pz6942', 'dst': '46'}
{'src': 'pz219', 'dst': '24'}
{'src': 'pz6953', 'dst': '46'}
{'src': '46', 'dst': 'pz6942'}
{'src': '46', 'dst': 'pz6953'}


  0%|          | 3/20000 [00:40<54:18:47,  9.78s/it]

...

  0%|          | 48/20000 [01:12<2:40:34,  2.07it/s]

{'src': 'pz6915', 'dst': '24'}
{'src': 'pz6942', 'dst': '46'}
{'src': 'pz6953', 'dst': '46'}
{'src': 'pz219', 'dst': '24'}
{'src': '46', 'dst': 'pz6942'}
{'src': '46', 'dst': 'pz6953'}
{'src': 'pz6915', 'dst': '24'}
{'src': 'pz6954', 'dst': '46'}
{'src': 'pz6951', 'dst': '24'}
{'src': 'pz6942', 'dst': '46'}
{'src': 'pz219', 'dst': '24'}


  0%|          | 49/20000 [01:42<51:37:16,  9.31s/it]

{'src': 'pz6953', 'dst': '46'}
{'src': 'pz219', 'dst': '24'}
{'src': '46', 'dst': 'pz6942'}
{'src': 'pz6915', 'dst': '24'}
{'src': '46', 'dst': 'pz6953'}
{'src': 'pz6942', 'dst': '46'}


  0%|          | 49/20000 [02:51<19:22:30,  3.50s/it]
